In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
from nltk.stem import *
import numpy as np
import re
from collections import defaultdict, Counter
from alphabet_detector import AlphabetDetector

In [59]:
porter = PorterStemmer()
snow = SnowballStemmer('russian')
PORTER, SNOW = [], []
#TAUTOS, LEMMAS, FIRST_SYMBOLS =[],[],[]
#PORTER_LEMMAS, SNOW_LEMMAS = [],[]
PORTER_LEFT, SNOW_LEFT = [],[]
#KEYWORDS = defaultdict(list)
SNOWS_SENT=[]
WORDS = []

In [60]:
def found_common(list_of_words, i):
    common = dict(Counter(list_of_words).most_common())
    to_append=False
    for n, c in common.items():
        if c>1 and n in SNOWS_SENT[i]:
            SNOW_LEFT.append(1.0)
            WORDS.append(n)
            to_append=True
            break
    if not to_append:
        WORDS.append(None)
        SNOW_LEFT.append(0.0)

In [61]:
def find_tauto(tokens, i, text_id):
    forms, porters, snows, first_symbols, lemmas, porter_lemmas, snow_lemmas = [],[],[],[],[],[],[]
    snows_sent=[]
    for tok in tokens:
        tok_text = tok.text.lower()
        grams = [x.strip()  for x in re.sub(r"</*br*>",',',tok['title']).split(',') if x!='']
        lemma = ''
        for gram in grams:
            if gram:
                if re.search('[а-я]', gram):
                    lemma = gram
                    break
        if set(grams).intersection({'S','ADV','A','V'}) and lemma not in KEYWORDS_2[text_id]:
            snows_sent.append(tok_text[:6])
#             lemmas.append(lemma)
#             porter_lemmas.append(porter.stem(lemma))
#             snow_lemmas.append(snow.stem(lemma))
#             forms.append(tok_text)
#             porters.append(porter.stem(tok_text))
#             snows.append(snow.stem(tok_text))
            first_symbols.append(tok_text[:6]) #загружать, загруженный; исследовать, исследование
    SNOWS_SENT.append(snows_sent)
    if i>=1:
        if i==1:
            if text_id==data['id'][i-1]:
                found_common(first_symbols+SNOWS_SENT[i-1], i)
            else: 
                found_common(first_symbols, i)
        if i>1:
            if text_id==data['id'][i-1] and text_id==data['id'][i-2]:
                found_common(first_symbols+SNOWS_SENT[i-1]+SNOWS_SENT[i-2],  i)
            elif text_id==data['id'][i-1] and not text_id==data['id'][i-2]:
                found_common(first_symbols+SNOWS_SENT[i-1],  i)
            else: 
                found_common(first_symbols,  i)
    else:
        found_common(first_symbols,  i)

In [6]:
data = pd.read_csv('tautos.csv')
data = data[pd.notnull(data['tagged'])]

In [7]:
tagged = data['tagged']
texts = data['text']
text_id = data['id']

In [12]:
for i, t in enumerate(tqdm(tagged)):
    soup = BeautifulSoup(t, 'html.parser')
    tokens = soup.find_all('span')
    for tok in tokens:
        tok_text = tok.text.lower()
        grams = [x.strip()  for x in re.sub(r"</*br*>",',',tok['title']).split(',') if x!='']
        lemma = ''
        for gram in grams:
            if gram:
                if re.search('[а-я]', gram):
                    lemma = gram
                    break
        if set(grams).intersection({'S','ADV','A','V'}): 
            KEYWORDS[text_id[i]].append(lemma)

100%|███████████████████████████████████████████████████████████████████████████| 51365/51365 [01:44<00:00, 493.88it/s]


In [13]:
KEYWORDS_2 = defaultdict(set)
# KEYWORDS_3 = defaultdict(set)
# KEYWORDS_4 = defaultdict(set)
# KEYWORDS_5 = defaultdict(set)
# KEYWORDS_6 = defaultdict(set)
for i, w in KEYWORDS.items():
    for n, v in Counter(w).most_common():
        if v > 2:
            KEYWORDS_2[i].add(n)
#         if v > 3:
#             KEYWORDS_3[i].add(n)
#         if v > 4:
#             KEYWORDS_4[i].add(n)
#         if v > 5:
#             KEYWORDS_5[i].add(n)
#         if v > 6:
#             KEYWORDS_6[i].add(n)
print(KEYWORDS_2)

defaultdict(<class 'set'>, {1: {'данный', 'навык', 'человек', 'коммерческий', 'задача', 'образование', 'то', 'знание', 'конкретный', 'нужный', 'гуманитарный', 'курс', 'курсы', 'получить', 'противник', 'наука', 'ответ', 'выгода', 'мнение', 'ценность', 'вопрос', 'студент', 'научить', 'духовный', 'быть', 'Константин', 'Иза', 'все', 'Сонин', 'считать', 'средний', 'только'}, 2: {'определить', 'краска', 'было', 'рисовать', 'Но', 'человек', 'нравиться', 'направление', 'все', 'необычный', 'дизайн', 'решить', 'изобразить', 'то'}, 3: {'навык', 'разный', 'человек', 'хотеть', 'материал', 'знание', 'развитие', 'нужный', 'гуманитарный', 'тяжелый', 'курс', 'курсы', 'а', 'ответ', 'больше', 'предмет', 'студент', 'быстро', 'важно', 'очень', 'быть', 'Константин', 'Сонин', 'только'}, 4: {'философия', 'человек', 'много', 'хотеть', 'знать', 'то', 'точный', 'нужный', 'гуманитарный', 'курс', 'курсы', 'дальнейший', 'а', 'помочь', 'наука', 'программа', 'страна', 'работать', 'жизнь', 'вопрос', 'хорошо', 'предмет

In [62]:
for i, t in enumerate(tqdm(tagged)):
    soup = BeautifulSoup(t, 'html.parser')
    tokens = soup.find_all('span')
    find_tauto(tokens,i,text_id[i])

100%|███████████████████████████████████████████████████████████████████████████| 51365/51365 [01:29<00:00, 572.10it/s]


In [63]:
print('acc={0}'.format(accuracy_score(data['tautos'], SNOW_LEFT)))
print(precision_recall_fscore_support(data['tautos'], SNOW_LEFT))

acc=0.8003699016840261
(array([0.99914881, 0.00263517]), array([0.80081087, 0.43548387]), array([0.88905239, 0.00523865]), array([51303,    62], dtype=int64))


In [64]:
tautos_df = pd.concat([data, pd.Series(WORDS).rename('common_words')], axis=1)
tautos_df = pd.concat([tautos_df, pd.Series(SNOW_LEFT).rename('first_symbols_left')], axis=1)

In [50]:
tautos_df.to_csv('whole_data.csv')

In [65]:
pos = tautos_df.loc[tautos_df['tautos']==1.0]

In [55]:
pos

,Unnamed: 0,Unnamed: 0.1,annotated,annotator_sentence.id,id,tagged,text,tag,tautos,common_words,first_symbols_left
9970,9970,9971,1,9972,332,"<span class=""token"" data-toggle=""tooltip"" titl...","Цель автора - донести читателю мысль о том, чт...","{'ref, pron', 'tauto'}",1.0,объясн,1.0
10275,10275,10276,1,10277,356,"<span class=""token"" data-toggle=""tooltip"" titl...","Он называет их Чёрными лебедями, сравнивая их ...","{'ref, pron', 'tauto'}",1.0,открыт,1.0
10285,10285,10286,1,10287,356,"<span class=""token"" data-toggle=""tooltip"" titl...",Для прочтения Вам не потребуется никаких допол...,"{'discourse, tauto', 'tauto'}",1.0,понятн,1.0
10574,10574,10575,1,10596,377,"<span class=""token"" data-toggle=""tooltip"" titl...","Все, что не укладывается в привычные и традици...","{'lex, word', 'discourse, tauto', 'tauto'}",1.0,None,0.0
11788,11788,11789,1,11810,480,"<span class=""token"" data-toggle=""tooltip"" titl...","Это лишь кусок всей экономической истории, но ...","{'lex, word', 'tauto'}",1.0,пробле,1.0
11936,11936,11975,1,11996,496,"<span class=""token"" data-toggle=""tooltip"" titl...",Информативная аннотация Фридрих фон Хайек обоз...,{'tauto'},1.0,влияни,1.0
12371,12371,12495,1,12516,534,"<span class=""token"" data-toggle=""tooltip"" titl...",В своем труде « Дорога к рабству » он утвержда...,"{'nmz', 'lex, word', 'tauto'}",1.0,None,0.0
13037,13037,13227,1,13248,573,"<span class=""token"" data-toggle=""tooltip"" titl...","Тем более, когда в нашей стране вдруг внезапно...",{'tauto'},1.0,изучен,1.0
13198,13198,13388,1,13409,577,"<span class=""token"" data-toggle=""tooltip"" titl...","На мой взгляд, сумасшедшая популярность данног...","{'styl, colloq', 'tauto'}",1.0,популя,1.0
13202,13202,13392,1,13413,577,"<span class=""token"" data-toggle=""tooltip"" titl...",В своей работе я попытаюсь рассмотреть определ...,"{'lack', 'lex, word', 'styl, tauto', 'ref, pro...",1.0,рассмо,1.0


In [66]:
pos = pos[['annotator_sentence.id','id','text','common_words','first_symbols_left']]

In [67]:
pos = pos.loc[pos['first_symbols_left']==0.0]
pos.to_csv('positive_first_symbols_left_not_found.csv')

In [68]:
neg = tautos_df.loc[(tautos_df['tautos']==0.0)&(tautos_df['first_symbols_left']==1.0)]
neg = neg[['annotator_sentence.id','id','text','common_words']]
neg.to_csv('negative_not_found.csv')

## Проблемы

Неоднозначность разметки: извлекается много случаев, являющихся тавтологией, не размеченных в КРУТе. Бывает, что одно и то же слово повторяется на протяжении всего текста (например, "студент" в первом тексте). Не учитывать слова, встречающиеся в тексте более n раз (это могут быть термины, ключевые слова) не спасает (полнота падает, точность практически не растет). Использовать машобуч из-за того, что многие положительные примеры не размечены, невозможно.

Субъективность: является ли тавтологией повторение собственных имен?